In [1]:
import numpy as np, pandas as pd

In [2]:
def run_filter(A, im_filter):
    """Run 2D cross-correlation ('valid' mode) of im_filter over A.
    
    Parameters
    ----------
    A : array_like, shape (H, W)
        Input 2D array (e.g., image or matrix).
    im_filter : array_like, shape (fh, fw)
        2D filter to slide over A. No flipping is performed (cross-correlation).
    
    Returns
    -------
    out : np.ndarray, shape (H - fh + 1, W - fw + 1)
        'Valid' cross-correlation result.
    """
    A = np.asarray(A)
    im_filter = np.asarray(im_filter)
    
    if A.ndim != 2 or im_filter.ndim != 2:
        raise ValueError("A and im_filter must both be 2D arrays.")
    
    H, W = A.shape
    fh, fw = im_filter.shape
    
    out_H = H - fh + 1
    out_W = W - fw + 1
    
    if out_H <= 0 or out_W <= 0:
        raise ValueError(
            f"Filter shape {im_filter.shape} is larger than input {A.shape} in at least one dimension."
        )
    
    out = np.zeros((out_H, out_W), dtype=float)
    
    # Cross-correlation: no flipping of im_filter
    for i in range(out_H):
        for j in range(out_W):
            window = A[i:i+fh, j:j+fw]
            out[i, j] = np.sum(window * im_filter)
    
    return out

In [3]:
# ------------------------
# Tests
# ------------------------

# 1) Sanity: 1x1 filter should return the same entries (valid mode)
A0 = np.array([[1, 2],
               [3, 4]])
f0 = np.array([[1]])
out0 = run_filter(A0, f0)
print("Test 1 — 1x1 filter (expect same as A):\n", out0)


# 2) Handout 3.1 — horizontal filter [-1, 0, 1] on 2x4 A
A = np.array([[-1,  1,  2,  7],
              [ 2,  6, -2,  1]])
f = np.array([[-1, 0, 1]])  # 1x3 row filter

out = run_filter(A, f)
expected = np.array([[ 3.,  6.],
                     [-4., -5.]])
print("Test 2 — handout 3.1 result:\n", out)


# 3) Horizontal 4-tap average (handout 3.2 interpreted horizontally)
# Use a 1x4 row filter of halves to slide across each row of A
f_row4 = np.array([[0.5, 0.5, 0.5, 0.5]])
out_row4 = run_filter(A, f_row4)
expected_row4 = np.array([[4.5],
                          [3.5]])
print("Test 3 — 1x4 horizontal average:\n", out_row4)


# 4) True vertical 4-tap average on a taller matrix (to demonstrate 4x1 column filter)
B = np.array([[ 1,  2,  3],
              [ 4,  5,  6],
              [ 7,  8,  9],
              [10, 11, 12],
              [13, 14, 15]])   # 5x3 (has 5 rows)
f_col4 = np.array([[0.5],
                   [0.5],
                   [0.5],
                   [0.5]])      # 4x1 column filter

out_col4 = run_filter(B, f_col4)
print("Test 4 — 4x1 vertical average on taller matrix:\n", out_col4)


# 5) Shape / large-filter error
try:
    run_filter(np.ones((2,2)), np.ones((1,3)))
    print("Test 5 failed: expected an error but none was raised.")
except ValueError as e:
    print("Test 5 — caught expected error:", e)

Test 1 — 1x1 filter (expect same as A):
 [[1. 2.]
 [3. 4.]]
Test 2 — handout 3.1 result:
 [[ 3.  6.]
 [-4. -5.]]
Test 3 — 1x4 horizontal average:
 [[4.5]
 [3.5]]
Test 4 — 4x1 vertical average on taller matrix:
 [[11. 13. 15.]
 [17. 19. 21.]]
Test 5 — caught expected error: Filter shape (1, 3) is larger than input (2, 2) in at least one dimension.
